<a href="https://colab.research.google.com/github/ChessEngineUS/hybrid-quantum-protein-folding/blob/main/notebooks/colab_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Quantum-AI Protein Folding - Quickstart

This notebook demonstrates the basic usage of the hybrid quantum-classical protein folding framework.

**What you'll learn:**
- Install the framework
- Predict protein structures
- Visualize results
- Use quantum simulators

**Runtime:** T4 GPU (recommended) or CPU


## Setup and Installation

First, let's install the framework and its dependencies.

In [ ]:
# Clone repository
!git clone https://github.com/ChessEngineUS/hybrid-quantum-protein-folding.git

# Change to repository directory
%cd hybrid-quantum-protein-folding

# Install dependencies
!pip install -q -r requirements.txt

# Install package
!pip install -q -e .

print('✅ Installation complete!')

## Imports

Import the necessary modules.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from hqpf.models import HybridModel

print('✅ Imports successful!')
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

## Predict Protein Structure

Let's predict the structure of a 10-residue peptide.

In [ ]:
# Define sequence
sequence_str = "ACDEFGHIKL"

# Amino acid to index mapping
aa_to_idx = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4,
    'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
    'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14,
    'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19
}

# Convert to tensor
sequence = torch.tensor([aa_to_idx[aa] for aa in sequence_str], dtype=torch.long)

print(f'Sequence: {sequence_str}')
print(f'Length: {len(sequence)} residues')

# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'\nUsing device: {device}')

model = HybridModel(
    n_residues=len(sequence),
    n_qubits=10,
    use_quantum=False,  # Use simulator
    device=device
)

print('Model initialized!')

# Predict structure
model.eval()
with torch.no_grad():
    outputs = model(sequence, n_candidates=10)

best_structure = outputs['best_structure']
best_energy = outputs['best_energy']

print(f'\n✅ Prediction complete!')
print(f'Best energy: {best_energy.item():.4f}')
print(f'Structure shape: {best_structure.shape}')

## Visualize Structure

Let's create a 3D visualization of the predicted structure.

In [ ]:
# Convert to numpy
structure_np = best_structure.cpu().numpy()

# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot backbone
ax.plot(structure_np[:, 0], structure_np[:, 1], structure_np[:, 2],
        'b-', linewidth=3, alpha=0.6, label='Backbone')

# Plot residues
scatter = ax.scatter(structure_np[:, 0], structure_np[:, 1], structure_np[:, 2],
                    c=range(len(sequence)), cmap='rainbow', s=200, alpha=0.8,
                    edgecolors='black', linewidths=2)

# Add labels
for i, (x, y, z) in enumerate(structure_np):
    ax.text(x, y, z, sequence_str[i], fontsize=14, fontweight='bold')

# Styling
ax.set_xlabel('X (Å)', fontsize=14, fontweight='bold')
ax.set_ylabel('Y (Å)', fontsize=14, fontweight='bold')
ax.set_zlabel('Z (Å)', fontsize=14, fontweight='bold')
ax.set_title(f'Predicted Structure: {sequence_str}\nEnergy: {best_energy.item():.4f}',
            fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
plt.colorbar(scatter, label='Residue Index', pad=0.1)

plt.tight_layout()
plt.show()

print('✅ Visualization complete!')

## Next Steps

Congratulations! You've successfully predicted and visualized a protein structure.

**Try these next:**
- Experiment with different sequences
- Increase `n_candidates` for better predictions
- Try the full tutorial notebook
- Train your own model

**Resources:**
- [Documentation](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding/tree/main/docs)
- [Examples](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding/tree/main/examples)
- [GitHub Repository](https://github.com/ChessEngineUS/hybrid-quantum-protein-folding)